In [ ]:
!pip install transformers
!pip install torch
!pip install shap


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 15.0 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import shap
import numpy as np
import scipy as sp
import torch.nn.functional as F

# Load BERT model and tokenizer
model_name = "/content/drive/MyDrive/legalbert"
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
model = BertForSequenceClassification.from_pretrained(model_name)


# Define a function that returns the predicted probabilities given text input for binary classification
def predictor(x):
    outputs = model(**tokenizer(x, return_tensors="pt", padding=True))
    probas = F.softmax(outputs.logits, dim=1).detach().numpy()
    val = sp.special.logit(probas[:, 1])
    return val


# allows you to efficiently process multiple inputs at once by avoiding redundant calculations and taking advantage of vectorized operations provided by NumPy.
def f_batch(x):
    val = np.array([])
    for i in x:
        val = np.append(val, predictor(i))
    return val

#Pink color words represent the words that are considered in overruling statement classification
#Blue color words indicates the words  that are not considered in overruling statemnt classification

explainer_bert = shap.Explainer(f_batch, tokenizer)

test = {'label': [1], 'text': ['accordingly, we approve the decision of the district court below and disapprove those of the courts in chessler, mitchell, and williams, to the extent that they conflict with this ruling.']}
shap_values = explainer_bert(test, fixed_context=1)
shap.plots.text(shap_values)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/legalbert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 100%|██████████| 2/2 [00:13<00:00, 13.35s/it]
